In [1]:
from lazypredict.Supervised import LazyRegressor
import numpy as np
import pandas as pd
import os
import pyarrow.feather as feather
from sklearn.model_selection import train_test_split

In [2]:
# input_root_path = "P:\SALMALIS Lab\Work Zone Dataset"
input_root_path = "E:\OneDrive - UBC\Work Zone\data"

In [4]:
df = feather.read_feather(os.path.join(input_root_path, 'TravelTimeInfo\TravelTime_HaveCellCoverage.ftr'))
df

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,Weather:Lat,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed
0,49831911074,Spread 3B,2022-05-03 17:59:55,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
1,49831911074,Spread 3B,2022-05-03 17:57:32,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
2,49831911074,Spread 3B,2022-05-03 17:55:23,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
3,49831911074,Spread 3B,2022-05-03 17:53:29,2.20,2.13,0.07,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
4,49831911074,Spread 3B,2022-05-03 17:51:45,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2816070,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:40:12,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.20,0.00,6.00,17-015W,True,617.33,2013.00,666.72,NaN
2816071,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:45:11,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.20,0.00,6.00,17-015W,True,617.33,2013.00,666.72,NaN
2816072,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:50:10,1.38,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.20,0.00,6.00,17-015W,True,617.33,2013.00,666.72,NaN
2816073,86704972236,AP 6-A-66-A • NB • Macro Spiecapag JV • 1073.6,2021-05-01 18:51:07,1.37,1.48,0.00,OK,2.45,NB,Spread 5B,...,49.24,14.20,0.00,6.00,17-015W,True,617.33,2013.00,666.72,NaN


In [6]:
df_assets = pd.read_csv(os.path.join(input_root_path, 'TravelTimeInfo\Assets_HaveCellCoverage_April_1-10.csv'))
df_assets

,49831911074
0,71808680042
1,53257039399
2,98626596943
3,88786298092
4,33388067973
...,...
121,12738620179
122,71201082738
123,25270109160
124,22374545720


In [13]:
df_features = pd.read_excel(os.path.join(input_root_path, 'Layout Plans\Weekly Approved H1080 TCPs\Work Zone Features.xlsx'), sheet_name=0)
df_features = df_features.drop(df_features.index[0]).reset_index(drop=True)
df_features

,WZ_ID,AssetNum,Direction,WZ_Hwy,Anticipated AADT (Veh/Day),Anticipated AADT Year,PostedSpd,WZ_IsBidirectional,WZ_ConstSignSpdLimDist,WZ_TransLenStart,...,WZ_NewLaneCount,WZ_ClosedLaneCount,WZ_IsMinAprch,WZ_BarrelSpace,Folder,Filename,Entered by,Additional Plan,Comments 1,Comments 2
0,AP 4-B-17-A,78385678679,NB,5.00,2872.00,2017.00,100,0,150,450,...,1,0,0,0,Weekly Approved H1080 TCPs\2022\April 4 - 8\Sp...,AP 4-B-17-A-TCPr TCP Drawing Rev0.pdf,Ahmed,NaN,NaN,NaN
1,AP 4-B-17-A,70353237409,SB,5.00,2872.00,2017.00,100,0,150,450,...,1,0,0,0,Weekly Approved H1080 TCPs\2022\April 4 - 8\Sp...,AP 4-B-17-A-TCPr TCP Drawing Rev0.pdf,Ahmed,NaN,NaN,NaN
2,AP 4-B-144-A,11297073233,NB,5.00,2872.00,2017.00,100,0,150,450,...,1,0,0,0,Weekly Approved H1080 TCPs\2022\April 4 - 8\Sp...,AP 4-B-144-A-TCPr TCP Drawing Rev0,Ahmed,NaN,NaN,NaN
3,AP 4-B-144-A,42238078130,SB,5.00,2872.00,2017.00,100,0,150,450,...,1,0,0,0,Weekly Approved H1080 TCPs\2022\April 4 - 8\Sp...,AP 4-B-144-A-TCPr TCP Drawing Rev0,Ahmed,NaN,NaN,NaN
4,AP 4-B-145-A,11297073233,NB,5.00,2872.00,2017.00,100,0,150,450,...,1,0,0,0,Weekly Approved H1080 TCPs\2022\April 4 - 8\Sp...,AP 4-B-145-A-TCPr TCP Drawing Rev0,Ahmed,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,AP 3-C-77-C,17960997656,SB,5.00,3151.00,2017.00,100,0,300,450,...,1,0,0,0,Weekly Approved H1080 TCPs\2022\3_March 14 - 1...,AP 3-C-77-B & AP 3-C-77-C FS-FLAGGER.pdf,Amit,NaN,NaN,NaN
221,AP 3-C-77-C,48287165966,NB,5.00,3151.00,2017.00,100,1,150,150,...,1,1,0,335,Weekly Approved H1080 TCPs\2022\3_March 14 - 1...,AP 3-C-77-B & AP 3-C-77-C SLAT-AFAD.pdf,Amit,NaN,NaN,NaN
222,AP 3-C-77-C,17960997656,SB,5.00,3151.00,2017.00,100,1,150,150,...,1,1,0,335,Weekly Approved H1080 TCPs\2022\3_March 14 - 1...,AP 3-C-77-B & AP 3-C-77-C SLAT-AFAD.pdf,Amit,NaN,NaN,NaN
223,AP 4-A-51-E,38283143270,NB,5.00,3255.00,2019.00,100,1,150,150,...,1,1,0,335,Weekly Approved H1080 TCPs\2022\3_March 14 - 1...,AP 4-A-51-E SLT-AFAD Revised.pdf,Amit,NaN,NaN,NaN


In [14]:
df_features.drop(df_features.columns[-6:], axis=1)

,WZ_ID,AssetNum,Direction,WZ_Hwy,Anticipated AADT (Veh/Day),Anticipated AADT Year,PostedSpd,WZ_IsBidirectional,WZ_ConstSignSpdLimDist,WZ_TransLenStart,...,WZ_SpdLim,WZ_IsTransSpd,WZ_IsPrepStop,WZ_ThxSignSpdLimDist,WZ_OrigLaneCount,WZ_IsOrigIDivided,WZ_NewLaneCount,WZ_ClosedLaneCount,WZ_IsMinAprch,WZ_BarrelSpace
0,AP 4-B-17-A,78385678679,NB,5.00,2872.00,2017.00,100,0,150,450,...,50,70,1,75,1,0,1,0,0,0
1,AP 4-B-17-A,70353237409,SB,5.00,2872.00,2017.00,100,0,150,450,...,50,70,1,75,1,0,1,0,0,0
2,AP 4-B-144-A,11297073233,NB,5.00,2872.00,2017.00,100,0,150,450,...,50,70,1,75,1,0,1,0,0,0
3,AP 4-B-144-A,42238078130,SB,5.00,2872.00,2017.00,100,0,150,450,...,50,70,1,75,1,0,1,0,0,0
4,AP 4-B-145-A,11297073233,NB,5.00,2872.00,2017.00,100,0,150,450,...,50,70,1,75,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,AP 3-C-77-C,17960997656,SB,5.00,3151.00,2017.00,100,0,300,450,...,50,70,1,75,1,0,1,0,0,0
221,AP 3-C-77-C,48287165966,NB,5.00,3151.00,2017.00,100,1,150,150,...,70,0,1,75,1,1,1,1,0,335
222,AP 3-C-77-C,17960997656,SB,5.00,3151.00,2017.00,100,1,150,150,...,70,0,1,75,1,1,1,1,0,335
223,AP 4-A-51-E,38283143270,NB,5.00,3255.00,2019.00,100,1,150,150,...,70,0,1,75,1,1,1,1,0,335


In [19]:
df_filtered = df[(91 <= df['CreatedDate:DayOfYear']) & (df['CreatedDate:DayOfYear']  <= 100) & (df['CreatedDate:Year'] == 2022)].reset_index(drop=True)
# pd.DataFrame(df_filtered['AssetNumber'].unique()).to_csv("E:\OneDrive - UBC\Work Zone\data\TravelTimeInfo\Assets_HaveCellCoverage_April_1-10.csv")
# df_filtered.to_csv("E:\OneDrive - UBC\Work Zone\data\TravelTimeInfo\TravelTime_HaveCellCoverage_April_1-10.csv")
df_filtered

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,Weather:Lat,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed
0,49831911074,Spread 3B,2022-04-04 17:58:45,2.25,2.13,0.12,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
1,49831911074,Spread 3B,2022-04-04 17:56:40,2.25,2.13,0.12,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
2,49831911074,Spread 3B,2022-04-04 17:54:48,2.25,2.13,0.12,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
3,49831911074,Spread 3B,2022-04-04 17:52:42,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
4,49831911074,Spread 3B,2022-04-04 17:50:48,2.23,2.13,0.10,OK,3.36,SB,Spread 3B,...,52.72,NaN,NaN,NaN,23-010NS,True,355.33,2017.00,373.10,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137572,40105790845,Spread 5B,2022-04-01 00:08:38,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00
137573,40105790845,Spread 5B,2022-04-01 00:06:37,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00
137574,40105790845,Spread 5B,2022-04-01 00:04:37,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00
137575,40105790845,Spread 5B,2022-04-01 00:02:37,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00


In [20]:
df_processed = df_filtered.dropna(axis=0).reset_index(drop=True).copy()
df_processed['CreatedDate:Season'] = df_processed['CreatedDate:Month']%12 // 3 + 1
df_processed

,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength,TravelDirect,Spread,...,Weather:Lat,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:SiteNo,Traffic:IsTemporary,Traffic:AADT,Traffic:BaseYear,Traffic:CalculatedAADT,MaxSpeed
0,19078590783,Spread 4B,2022-04-08 18:09:33,1.62,1.63,0.00,OK,2.70,NB,Spread 4B,...,51.65,7.00,0.00,9.00,21-013EW,True,55.43,2017.00,58.20,100.00
1,19078590783,Spread 4B,2022-04-08 18:07:46,1.62,1.63,0.00,OK,2.70,NB,Spread 4B,...,51.65,7.00,0.00,9.00,21-013EW,True,55.43,2017.00,58.20,100.00
2,19078590783,Spread 4B,2022-04-08 18:05:54,1.62,1.63,0.00,OK,2.70,NB,Spread 4B,...,51.65,7.00,0.00,9.00,21-013EW,True,55.43,2017.00,58.20,100.00
3,19078590783,Spread 4B,2022-04-08 18:04:01,1.62,1.63,0.00,OK,2.70,NB,Spread 4B,...,51.65,7.00,0.00,9.00,21-013EW,True,55.43,2017.00,58.20,100.00
4,19078590783,Spread 4B,2022-04-08 18:02:07,1.62,1.63,0.00,OK,2.70,NB,Spread 4B,...,51.65,7.00,0.00,9.00,21-013EW,True,55.43,2017.00,58.20,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48695,40105790845,Spread 5B,2022-04-01 00:08:38,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00
48696,40105790845,Spread 5B,2022-04-01 00:06:37,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00
48697,40105790845,Spread 5B,2022-04-01 00:04:37,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00
48698,40105790845,Spread 5B,2022-04-01 00:02:37,2.13,1.28,0.85,OK,2.21,NB,Spread 5B,...,49.24,6.10,0.00,2.00,17-044EW,True,41.00,2016.00,43.46,100.00


In [21]:
x_drop_cols = ['TravelDirect', 'Spread', 'TrafficImpact', 'RoadType', 'TrafficControlSetup',
               'AssetIdentifier', 'CreatedDate', 'State', 'CreatedDate:Date', 'CreatedDate:Year', 'Weather:StationID', 'Weather:Long', 'Weather:Lat', 'Traffic:SiteNo', 'Traffic:IsTemporary', 'Traffic:AADT', 'Traffic:BaseYear', 'MaxSpeed']
df_x = df_processed.drop(columns=x_drop_cols)
df_y = df_processed['DelayTime']

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=.2, random_state=42)
x_train


,AssetNumber,UnderConstructionTime,BaselineTime,DelayTime,SegmentLength,CreatedDate:Weekday,CreatedDate:DayOfYear,CreatedDate:Month,CreatedDate:Day,CreatedDate:Hour,CreatedDate:MinuteInDay,CreatedDate:Season,Weather:Temp,Weather:Precip,Weather:WindSpd,Traffic:CalculatedAADT
12692,23023696729,1.30,1.25,0.05,2.18,2,96,4,6,1,78,2,3.10,0.00,3.00,24.38
25846,98128214373,1.92,1.43,0.49,2.47,1,95,4,5,0,20,2,7.20,0.00,18.00,43.46
11911,23023696729,1.23,1.25,0.00,2.18,4,98,4,8,1,112,2,11.90,0.00,5.00,24.38
33645,12738620179,1.32,1.28,0.04,2.15,4,98,4,8,22,1374,2,7.30,0.00,7.00,104.94
9398,60041619645,1.67,1.73,0.00,2.86,2,96,4,6,19,1195,2,11.10,0.00,11.00,215.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,23023696729,1.28,1.25,0.03,2.18,6,100,4,10,21,1294,2,2.50,0.00,3.00,138.86
44732,22374545720,1.50,1.42,0.08,2.43,0,94,4,4,2,134,2,5.10,0.70,5.00,22.26
38158,71201082738,1.32,1.22,0.10,2.14,1,95,4,5,2,154,2,6.80,0.00,18.00,22.26
860,64413418329,1.57,1.67,0.00,2.70,4,98,4,8,10,654,2,14.60,0.00,22.00,105.70


In [6]:
reg = LazyRegressor(predictions=True)
models, predictions = reg.fit(x_train, x_test, y_train, y_test)

 62%|██████▏   | 26/42 [37:01:42<22:47:12, 5127.01s/it]


KeyboardInterrupt: 

In [ ]:
predictions